# Basic linear algebra in Julia
Originally started by : Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)
with edits from Jane Herriman

First let's define a random matrix

In [1]:
A = rand(1:4,3,3)

3×3 Array{Int64,2}:
 2  2  3
 1  3  1
 4  2  2

Define a vector of ones

In [2]:
x = fill(1.0, (3,)) # = fill(1.0, 3)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type Array{Int64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2}. 

Many of the basic operations are the same as in other languages
#### Multiplication

In [3]:
b = A*x

3-element Array{Float64,1}:
 7.0
 5.0
 8.0

#### Transposition
As in other languages `A'` is the conjugate transpose, or adjoint

In [4]:
A'

3×3 LinearAlgebra.Adjoint{Int64,Array{Int64,2}}:
 2  1  4
 2  3  2
 3  1  2

and we can get the transpose with

In [5]:
transpose(A)

3×3 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 2  1  4
 2  3  2
 3  1  2

#### Transposed multiplication
Julia allows us to write this without *

In [6]:
A'A

3×3 Array{Int64,2}:
 21  15  15
 15  17  13
 15  13  14

#### Solving linear systems 
The problem $Ax=b$ for ***square*** $A$ is solved by the \ function.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 0.9999999999999999

`A\b` gives us the *least squares solution* if we have an overdetermined linear system (a "tall" matrix)

In [8]:
Atall = rand(3, 2)

3×2 Array{Float64,2}:
 0.476765   0.959126
 0.423559   0.934104
 0.0815409  0.00310295

In [9]:
Atall\b

2-element Array{Float64,1}:
  93.20315518162505
 -37.99745295762181

and the *minimum norm least squares solution* if we have a rank-deficient least squares problem

In [10]:
v = rand(3)
rankdef = hcat(v, v)

3×2 Array{Float64,2}:
 0.887259  0.887259
 0.127225  0.127225
 0.113116  0.113116

In [11]:
rankdef\b

2-element Array{Float64,1}:
 4.7486951933721775
 4.748695193372178

Julia also gives us the minimum norm solution when we have an underdetermined solution (a "short" matrix)

In [12]:
bshort = rand(2)
Ashort = rand(2, 3)

2×3 Array{Float64,2}:
 0.920981  0.452076  0.992689
 0.347255  0.490086  0.343507

In [13]:
Ashort\bshort

3-element Array{Float64,1}:
 -0.0914521847895992
  0.7776806442335451
 -0.17780127900646162

4x + 3y = 10

x + y = 3

In [14]:
A = [[4 3];[1 1]]
b = [10,3]
x = A\b

2-element Array{Float64,1}:
 1.0
 2.0

# The LinearAlgebra library

While much of linear algebra is available in Julia by default (as shown above), there's a standard library named `LinearAlgebra` that brings in many more relevant names and functions. In particular, it provides factorizations and some structured matrix types.  As with all packages, you can bring these additional features into your session with a `using LinearAlgebra`.

In [15]:
using Pkg
Pkg.add("LinearAlgebra")

   Updating registry at `C:\Users\Victor\.julia\registries\General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [================

Fetching: [========================================>]  100.0 %            ]  66.2 %==============================>          ]  74.6 %94.8 %

  Resolving package versions...
  Installed Zlib_jll ─ v1.2.11+12
   Updating `C:\Users\Victor\.julia\environments\v1.4\Project.toml`
  [37e2e46d] + LinearAlgebra 
   Updating `C:\Users\Victor\.julia\environments\v1.4\Manifest.toml`
  [83775a58] ↑ Zlib_jll v1.2.11+11 ⇒ v1.2.11+12


To list the various functions available in the package we use the `names` command

In [17]:
using LinearAlgebra

In [18]:
names(LinearAlgebra)

124-element Array{Symbol,1}:
 :/
 :Adjoint
 :BLAS
 :Bidiagonal
 :BunchKaufman
 :Cholesky
 :CholeskyPivoted
 :Diagonal
 :Eigen
 :Factorization
 :GeneralizedEigen
 :GeneralizedSVD
 :GeneralizedSchur
 ⋮
 :svdvals
 :svdvals!
 :sylvester
 :tr
 :transpose
 :transpose!
 :tril
 :tril!
 :triu
 :triu!
 :×
 :⋅

To gather information of all the current variables in the environment use `varinfo`

In [19]:
varinfo()

| name    |     size | summary                    |
|:------- | --------:|:-------------------------- |
| A       | 72 bytes | 2×2 Array{Int64,2}         |
| Ashort  | 88 bytes | 2×3 Array{Float64,2}       |
| Atall   | 88 bytes | 3×2 Array{Float64,2}       |
| Base    |          | Module                     |
| Core    |          | Module                     |
| Main    |          | Module                     |
| b       | 56 bytes | 2-element Array{Int64,1}   |
| bshort  | 56 bytes | 2-element Array{Float64,1} |
| rankdef | 88 bytes | 3×2 Array{Float64,2}       |
| v       | 64 bytes | 3-element Array{Float64,1} |
| x       | 56 bytes | 2-element Array{Float64,1} |


#### How do I delete an object in memory? ####

Julia does not have an analog of MATLAB's `clear` function; once a name is defined in a Julia session (technically, in module `Main`), it is always present.

If memory usage is your concern, you can always replace objects with ones that consume less memory. For example, if `A` is a gigabyte-sized array that you no longer need, you can free the memory with `A = nothing`. The memory will be released the next time the garbage collector runs; you can force this to happen with `gc()`. Moreover, an attempt to use `A` will likely result in an error, because most methods are not defined on type `Nothing`.

#### How can I modify the declaration of a type in my session?

Perhaps you've defined a type and then realize you need to add a new field. If you try this, you get the error:

`ERROR: invalid redefinition of constant MyType`

Types in module `Main` cannot be redefined.

### Exercises

#### 11.1 
Take the inner product (or "dot" product) of a vector `v` with itself and assign it to variable `dot_v`.



In [20]:
v

3-element Array{Float64,1}:
 0.8872592205261467
 0.12722548147925816
 0.11311592937782167

In [21]:
using LinearAlgebra

In [24]:
dot_v=dot(v,v)

0.8162104610253028

In [25]:
@assert dot_v == 14

AssertionError: AssertionError: dot_v == 14

#### 11.2 
Take the outer product of a vector v with itself and assign it to variable `cross_v`

In [26]:
?cross

search: cross count_zeros copy_transpose!



```
cross(x, y)
×(x,y)
```

Compute the cross product of two 3-vectors.

# Examples

```jldoctest
julia> a = [0;1;0]
3-element Array{Int64,1}:
 0
 1
 0

julia> b = [0;0;1]
3-element Array{Int64,1}:
 0
 0
 1

julia> cross(a,b)
3-element Array{Int64,1}:
 1
 0
 0
```


In [27]:
cross_v=cross(v,v)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [28]:
@assert cross_v == [0, 0, 0]

Please execute the @assert cell once you are done with the exercises to validate your solution.